In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [3]:
train_dataset = datasets.CIFAR10(root='./data',
                                 train = True,
                                 transform = ToTensor(),
                                 download = True)

test_dataset = datasets.CIFAR10(root = './data',
                                train = False,
                                transform = ToTensor(),
                                download = True)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
batch_size = 256
num_classes = 10
learning_rate = 0.01
num_epochs = 20

In [6]:
train_loader = DataLoader(dataset = train_dataset,
                          batch_size = batch_size,
                          shuffle = True)

test_loader = DataLoader(dataset = test_dataset,
                         batch_size = batch_size,
                         shuffle = True)

In [42]:
class ConvNeuralNet(nn.Module):
  def __init__(self, num_classes):
    super(ConvNeuralNet, self).__init__()
    self.conv_l1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2)
    self.max_pool1 = nn.MaxPool2d(kernel_size=4,stride=3)
    
    self.conv_l2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=2)
    self.max_pool2 = nn.MaxPool2d(kernel_size=4, stride=3)
    self.fc1 = nn.Linear(128,256)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(256,num_classes)

  def forward(self, x):
    out = self.conv_l1(x)
    out = self.max_pool1(out)
    out = self.conv_l2(out)
    
    out = self.max_pool2(out)

    out = out.reshape(out.size(0), -1)

    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    return out

In [43]:
device = torch.device('cpu')

In [44]:
model = ConvNeuralNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay = 0.005, momentum = 0.9)
total_step = len(train_loader)

In [45]:
acc=[0,0]
k = 0
j = 0
while True:
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i/195==1:
      j+=1
      with torch.no_grad():
        correct = 0
        samples = 0
        for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _,predicted = torch.max(outputs.data,1)
          samples+=labels.size(0)
          correct+= (predicted == labels).sum().item()
      print('Epoch [{}], loss:{:.4f}'.format(j,loss.item()))
      print('accuracy = ',100*(correct/samples))
      print('\n\n')
      accuracy = correct/samples
      acc+=[accuracy]
      if acc[len(acc)-1]<acc[len(acc)-2] and acc[len(acc)-2]<acc[len(acc)-3]:
        k+=1
        break
  if k==1:
    break

plt.plot(acc[2:])

Epoch [1], loss:2.0756
accuracy =  23.990000000000002



Epoch [2], loss:1.9820
accuracy =  30.09



Epoch [3], loss:1.8225
accuracy =  34.27



Epoch [4], loss:1.5076
accuracy =  38.66



Epoch [5], loss:1.6492
accuracy =  39.83



Epoch [6], loss:1.6206
accuracy =  41.29



Epoch [7], loss:1.4239
accuracy =  44.29



Epoch [8], loss:1.4748
accuracy =  44.54



Epoch [9], loss:1.4709
accuracy =  46.7



Epoch [10], loss:1.4104
accuracy =  47.32



Epoch [11], loss:1.3461
accuracy =  48.44



Epoch [12], loss:1.6745
accuracy =  49.27



Epoch [13], loss:1.5000
accuracy =  50.19



Epoch [14], loss:1.2449
accuracy =  50.690000000000005



Epoch [15], loss:1.3748
accuracy =  51.68000000000001



Epoch [16], loss:1.4621
accuracy =  52.62



Epoch [17], loss:1.3522
accuracy =  52.09



Epoch [18], loss:1.2123
accuracy =  53.64



Epoch [19], loss:1.1428
accuracy =  53.66



Epoch [20], loss:1.2335
accuracy =  53.16



Epoch [21], loss:1.1678
accuracy =  53.32



Epoch [22], loss:1.1020
acc

KeyboardInterrupt: 

In [46]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [47]:
model = ConvNeuralNet(num_classes).to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [53]:
with torch.no_grad():
        correct = 0
        samples = 0
        for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _,predicted = torch.max(outputs.data,1)
          samples+=labels.size(0)
          correct+= (predicted == labels).sum().item()
print('accuracy = ',100*(correct/samples))

accuracy =  57.56
